In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import gpytorch
# from voltron.robinhood_utils import GetStockData
import os
# import robin_stocks.robinhood as r
import pickle5 as pickle

import sys
sys.path.append("../")
from voltron.likelihoods import VolatilityGaussianLikelihood
from voltron.kernels import BMKernel, VolatilityKernel
from voltron.models import BMGP, VoltronGP, SingleTaskVariationalGP
from voltron.means import LogLinearMean
from gpytorch.kernels import ScaleKernel, RBFKernel, MaternKernel

Warning no robinhood utils.


In [2]:
sns.set_style('white')
# style.use('whitegrid')
palette = ["#1b4079", "#C6DDF0", "#048A81", "#B9E28C", "#8C2155", "#AF7595", "#E6480F", "#FA9500"]
sns.set(palette = palette, font_scale=2.0, style="white", rc={"lines.linewidth": 4.0})

In [3]:
torch.random.manual_seed(200)

In [4]:
span = "5year"
interval = 'day'
T = 5.

In [5]:
with open("../../spdr-data/XLF.pkl", "rb") as handle:
    raw_data = pickle.load(handle)
    
tckrs = np.unique(raw_data["symbol"])

In [6]:
data_list = []
for i, tckr in enumerate(tckrs):
    data = raw_data[raw_data["symbol"] == tckr]

    y = torch.FloatTensor(data['close_price'].to_numpy())
    data_list.append(y)

ts = torch.linspace(0, T, data.shape[0]) + 1
dt = ts[1] - ts[0]
full_data = torch.stack(data_list)

In [7]:
full_returns = torch.log(full_data[..., 1:] / full_data[..., :-1])

In [8]:
device = torch.device("cuda:6")
torch.cuda.set_device(device)

In [39]:
def get_and_fit_gpcv(x, log_returns):
    train_x = x#[:-1]
    # prepare model
    likelihood = VolatilityGaussianLikelihood(param="exp")
    # likelihood.raw_a.data -= 6.
    covar_module = BMKernel()
    model = SingleTaskVariationalGP(
        init_points=train_x.view(-1,1), 
        likelihood=likelihood, 
        use_piv_chol_init=False,
        mean_module = gpytorch.means.ConstantMean(), 
        covar_module=covar_module, 
        learn_inducing_locations=False,
        use_whitened_var_strat=False,
    )
    # model.mean_module.constant.data -= 4.
    model.initialize_variational_parameters(likelihood, train_x, y=log_returns)
    
    model = model.to(device)
    
    import os
    smoke_test = ('CI' in os.environ)
    training_iterations = 2 if smoke_test else 500


    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    # likelihood parameters should be taken acct of in the model
    optimizer = torch.optim.Adam([
        {"params": model.parameters()}, 
        # {"params": likelihood.parameters(), "lr": 0.1}
    ], lr=0.01)

    # "Loss" for GPs - the marginal log likelihood
    # num_data refers to the number of training datapoints
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, log_returns.numel())
    
    old_loss = 10000.
    print_every = 50
    for i in range(training_iterations):
        # Zero backpropped gradients from previous iteration
        optimizer.zero_grad()
        # Get predictive output
        with gpytorch.settings.num_gauss_hermite_locs(75):
            output = model(train_x)
            # Calc loss and backprop gradients
            loss = -mll(output, log_returns)
            loss.backward()
            if i % print_every == 0:
                print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
            optimizer.step()
        if old_loss <= loss and i > 100:
            print(old_loss, loss)
            break
        else:
            old_loss = loss.item()
            
    model.eval();
    likelihood.eval();
    predictive = model(x)
    pred_scale = likelihood(predictive).scale.mean(0).detach()
    samples = likelihood(predictive).scale.detach()
    
#     plt.plot(x.cpu(), pred_scale.cpu(), linewidth = 4)
#     plt.plot(x.cpu(), samples.t().cpu(), color = "gray", alpha = 0.3)
#     # plt.ylim((0, 0.25))
#     plt.show()
    
    # return scaled volatility prediction
    return pred_scale / dt**0.5
    

def get_and_fit_vol_model(train_x, est_vol):
    vol_lh = gpytorch.likelihoods.GaussianLikelihood()
    vol_lh.noise.data = torch.tensor([1e-6])
    vol_model = BMGP(train_x, est_vol.log(), vol_lh)
    vol_model = vol_model.to(device)
    

    optimizer = torch.optim.Adam([
        {'params': vol_model.parameters()},  # Includes GaussianLikelihood parameters
    ], lr=0.01)

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(vol_lh, vol_model)
    old_loss = 10000
    for i in range(500):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = vol_model(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, est_vol.log())
        loss.backward()
        if i % 50 == 0:
            print(loss.item())
        optimizer.step()
#         if old_loss <= loss:
#             break
#         else:
#             old_loss = loss.item()
        
    return vol_model

def get_and_fit_data_model(train_x, train_y, pred_vol, vol_model):
    voltron_lh = gpytorch.likelihoods.GaussianLikelihood()
    voltron = VoltronGP(train_x, train_y.log(), voltron_lh, pred_vol)
    # voltron.mean_module = gpytorch.means.LinearMean(1)
    voltron.mean_module = LogLinearMean(1)
    voltron.mean_module.initialize_from_data(train_x, train_y.log())
    voltron.likelihood.raw_noise.data = torch.tensor([1e-6])
    voltron.vol_lh = vol_model.likelihood
    voltron.vol_model = vol_model
    voltron = voltron.to(device)

    grad_flags = [False, True, True, True, False, False, False]

    for idx, p in enumerate(voltron.parameters()):
        p.requires_grad = grad_flags[idx]

    voltron.train();
    voltron_lh.train();
    voltron.vol_lh.train();
    voltron.vol_model.train();

    # Use the adam optimizer
    optimizer = torch.optim.Adam([
        {'params': voltron.parameters()},  # Includes GaussianLikelihood parameters
    ], lr=0.1)

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(voltron_lh, voltron)

    for i in range(500):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = voltron(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, train_y.log())
        loss.backward()
        # print(loss.item())
        optimizer.step()
    return voltron

def predict_prices(test_x, voltron, nvol=10, npx=10):
    ntest = test_x.shape[0]
    vol_paths = torch.zeros(nvol, ntest)
    px_paths = torch.zeros(npx*nvol, ntest)

    voltron.vol_model.eval();
    voltron.eval();

    for vidx in range(nvol):
        vol_pred = voltron.vol_model(test_x).sample().exp()
        vol_paths[vidx, :] = vol_pred.detach()

        px_pred = voltron.GeneratePrediction(test_x, vol_pred, npx).exp()
        px_paths[vidx*npx:(vidx*npx+npx), :] = px_pred.detach().T
    return px_paths, vol_paths

In [40]:
def run_models_for_paths(train_x, train_y, log_returns, test_x):
    all_gpcv_paths, all_gpcv_vols = [], []
    for i in range(len(tckrs)):
        print("now running ", tckrs[i])

        pred_vol = get_and_fit_gpcv(train_x, log_returns[i])
        vol_model = get_and_fit_vol_model(train_x, pred_vol)
        data_model = get_and_fit_data_model(train_x, train_y[i], pred_vol, vol_model)
        paths, vols = predict_prices(test_x, data_model)
        all_gpcv_paths.append(paths.detach())
        all_gpcv_vols.append(vols.detach())
        
    return torch.stack(all_gpcv_paths), torch.stack(all_gpcv_vols)

In [41]:
train_splits = list(range(252, data.shape[0], int(252/2)))
eval_splits = list(range(252+int(252/2), data.shape[0], int(252/2))) + [-1]
print(len(train_splits), len(eval_splits))

8 8


In [42]:
train_splits.insert(0, 0)

In [43]:
ts = ts.to(device)
full_returns = full_returns.to(device)
full_data = full_data.to(device)

In [44]:
ts = ts[1:]

In [45]:
x_list = []
v_list = []
for i in range(len(eval_splits)):
    
    torch.cuda.empty_cache()
    
    train_start = train_splits[i]
    train_end = train_splits[i+1]
    test_end = eval_splits[i]
    print("Now running ", train_start, train_end)
    
    x_paths, v_paths = run_models_for_paths(
        ts[train_start:train_end], 
        full_data[:, train_start:train_end],
        full_returns[:, train_start:train_end],
        ts[train_end:test_end]
    )
    
    x_list.append(x_paths)
    v_list.append(v_paths)

Now running  0 252
now running  AIG
Using gp-exp parameterization.
Iter 1/500 - Loss: 13.153
Iter 51/500 - Loss: -3.029
Iter 101/500 - Loss: -3.155
Iter 151/500 - Loss: -3.174
-3.1779394149780273 tensor(-3.1779, device='cuda:6', grad_fn=<NegBackward>)
0.788555383682251
0.5886960029602051
0.3734378218650818
0.1395917683839798
-0.1050044521689415
-0.35043373703956604
-0.5921759605407715
-0.8283711075782776
-1.0562853813171387
-1.2732905149459839
now running  AON
Using gp-exp parameterization.
Iter 1/500 - Loss: 12.693
Iter 51/500 - Loss: -3.320
Iter 101/500 - Loss: -3.419
Iter 151/500 - Loss: -3.432
-3.433570384979248 tensor(-3.4336, device='cuda:6', grad_fn=<NegBackward>)
0.7922147512435913
0.5891477465629578
0.3729459047317505
0.13836640119552612
-0.10922157019376755
-0.3605894148349762
-0.6080528497695923
-0.8489323258399963
-1.0809381008148193
-1.3013389110565186
now running  AXP
Using gp-exp parameterization.
Iter 1/500 - Loss: 11.654
Iter 51/500 - Loss: -3.131
Iter 101/500 - Loss: 

0.5860093235969543
0.3690284490585327
0.13110987842082977
-0.1187770888209343
-0.3686058521270752
-0.6130086779594421
-0.8517631888389587
-1.0838119983673096
-1.3056215047836304
now running  PGR
Using gp-exp parameterization.
Iter 1/500 - Loss: 13.388
Iter 51/500 - Loss: -3.158
Iter 101/500 - Loss: -3.249
Iter 151/500 - Loss: -3.264
Iter 201/500 - Loss: -3.270
-3.270717144012451 tensor(-3.2707, device='cuda:6', grad_fn=<NegBackward>)
0.7928061485290527
0.5900781154632568
0.374180406332016
0.14033915102481842
-0.105836421251297
-0.35498982667922974
-0.5995356440544128
-0.8372047543525696
-1.0662400722503662
-1.284266710281372
now running  PNC
Using gp-exp parameterization.
Iter 1/500 - Loss: 12.733
Iter 51/500 - Loss: -2.934
Iter 101/500 - Loss: -3.000
Iter 151/500 - Loss: -3.005
-3.0054173469543457 tensor(-3.0054, device='cuda:6', grad_fn=<NegBackward>)
0.7791319489479065
0.5811432600021362
0.36264365911483765
0.12264657765626907
-0.12776342034339905
-0.3796503245830536
-0.628573596477

now running  C
Using gp-exp parameterization.
Iter 1/500 - Loss: 14.278
Iter 51/500 - Loss: -2.815
Iter 101/500 - Loss: -2.899
Iter 151/500 - Loss: -2.915
Iter 201/500 - Loss: -2.920
-2.920696973800659 tensor(-2.9207, device='cuda:6', grad_fn=<NegBackward>)
0.8141818642616272
0.6098836660385132
0.3936675786972046
0.16313856840133667
-0.07701117545366287
-0.32098838686943054
-0.5630947351455688
-0.7989878058433533
-1.025909662246704
-1.2414876222610474
now running  CB
Using gp-exp parameterization.
Iter 1/500 - Loss: 13.280
Iter 51/500 - Loss: -2.990
Iter 101/500 - Loss: -3.058
Iter 151/500 - Loss: -3.064
-3.064544677734375 tensor(-3.0645, device='cuda:6', grad_fn=<NegBackward>)
0.7982655167579651
0.5962380766868591
0.3799229860305786
0.14452356100082397
-0.1038779690861702
-0.35508403182029724
-0.6017657518386841
-0.8420687913894653
-1.0733909606933594
-1.2926477193832397
now running  CME
Using gp-exp parameterization.
Iter 1/500 - Loss: 13.593
Iter 51/500 - Loss: -2.820
Iter 101/500 -

0.5998151302337646
0.38458696007728577
0.15229356288909912
-0.09125198423862457
-0.33747828006744385
-0.5801182389259338
-0.8161746263504028
-1.0428998470306396
-1.2576375007629395
now running  TROW
Using gp-exp parameterization.
Iter 1/500 - Loss: 14.190
Iter 51/500 - Loss: -2.749
Iter 101/500 - Loss: -2.839
Iter 151/500 - Loss: -2.851
-2.852292537689209 tensor(-2.8523, device='cuda:6', grad_fn=<NegBackward>)
0.8089371919631958
0.6060937643051147
0.3904849588871002
0.1597355455160141
-0.08134309202432632
-0.3264242708683014
-0.5696443915367126
-0.8071238398551941
-1.0361913442611694
-1.2543001174926758
now running  TRV
Using gp-exp parameterization.
Iter 1/500 - Loss: 13.522
Iter 51/500 - Loss: -2.858
Iter 101/500 - Loss: -2.951
Iter 151/500 - Loss: -2.964
-2.96557879447937 tensor(-2.9656, device='cuda:6', grad_fn=<NegBackward>)
0.7989947199821472
0.5981462001800537
0.38304316997528076
0.15003235638141632
-0.09394945949316025
-0.3392218053340912
-0.5806906819343567
-0.8171257376670837

Iter 51/500 - Loss: -2.859
Iter 101/500 - Loss: -2.979
Iter 151/500 - Loss: -3.006
Iter 201/500 - Loss: -3.016
-3.0189549922943115 tensor(-3.0190, device='cuda:6', grad_fn=<NegBackward>)
0.7880347967147827
0.5909725427627563
0.37352144718170166
0.1342746913433075
-0.11587246507406235
-0.36440908908843994
-0.6046760678291321
-0.8311443328857422
-1.044066309928894
-1.2519936561584473
now running  MET
Using gp-exp parameterization.
Iter 1/500 - Loss: 15.322
Iter 51/500 - Loss: -2.693
Iter 101/500 - Loss: -2.812
Iter 151/500 - Loss: -2.831
Iter 201/500 - Loss: -2.837
-2.8373968601226807 tensor(-2.8374, device='cuda:6', grad_fn=<NegBackward>)
0.7849217653274536
0.5888321995735168
0.37008845806121826
0.12998630106449127
-0.1188189834356308
-0.36612316966056824
-0.6060079336166382
-0.8357622027397156
-1.06046462059021
-1.2785383462905884
now running  MMC
Using gp-exp parameterization.
Iter 1/500 - Loss: 14.134
Iter 51/500 - Loss: -3.175
Iter 101/500 - Loss: -3.252
Iter 151/500 - Loss: -3.260


...

Iter 51/500 - Loss: -1.720
Iter 101/500 - Loss: -2.035
Iter 151/500 - Loss: -2.168
Iter 201/500 - Loss: -2.235
Iter 251/500 - Loss: -2.271
Iter 301/500 - Loss: -2.293
Iter 351/500 - Loss: -2.307
Iter 401/500 - Loss: -2.317
Iter 451/500 - Loss: -2.325
0.9056083559989929
0.7008067965507507
0.4751582741737366
0.2370198667049408
-0.004062592051923275
-0.24032218754291534
-0.46601083874702454
-0.6770763397216797
-0.8710703253746033
-1.0475211143493652
now running  MET
Using gp-exp parameterization.
Iter 1/500 - Loss: 22.977
Iter 51/500 - Loss: -1.571
Iter 101/500 - Loss: -1.955
Iter 151/500 - Loss: -2.119
Iter 201/500 - Loss: -2.207
Iter 251/500 - Loss: -2.256
Iter 301/500 - Loss: -2.285
Iter 351/500 - Loss: -2.304
Iter 401/500 - Loss: -2.318
Iter 451/500 - Loss: -2.328
0.9170172810554504
0.7084949016571045
0.4800128638744354
0.24026302993297577
-0.001887730322778225
-0.2391265481710434
-0.4657772183418274
-0.6777106523513794
-0.8723911643028259
-1.0492671728134155
now running  MMC
Using gp

-0.4722099006175995
-0.6844049096107483
-0.8805955648422241
-1.0607174634933472
Now running  882 1008
now running  AIG
Using gp-exp parameterization.
Iter 1/500 - Loss: 14.062
Iter 51/500 - Loss: -1.815
Iter 101/500 - Loss: -1.901
Iter 151/500 - Loss: -1.918
-1.9215914011001587 tensor(-1.9216, device='cuda:6', grad_fn=<NegBackward>)
0.7816594839096069
0.5952906608581543
0.3857024908065796
0.15740616619586945
-0.08198172599077225
-0.32550376653671265
-0.5678163766860962
-0.8050296306610107
-1.0341418981552124
-1.252485990524292
now running  AON
Using gp-exp parameterization.
Iter 1/500 - Loss: 13.537
Iter 51/500 - Loss: -2.522
Iter 101/500 - Loss: -2.639
Iter 151/500 - Loss: -2.673
Iter 201/500 - Loss: -2.688
-2.691758632659912 tensor(-2.6917, device='cuda:6', grad_fn=<NegBackward>)
0.7777727842330933
0.5852289199829102
0.3697584569454193
0.14098650217056274
-0.09771644324064255
-0.34170085191726685
-0.5850217938423157
-0.8234906196594238
-1.0538995265960693
-1.2734280824661255
now runn

0.5907625555992126
0.3813033699989319
0.1540694534778595
-0.08402034640312195
-0.3259555995464325
-0.566365122795105
-0.801283597946167
-1.0275434255599976
-1.2422423362731934
now running  MSCI
Using gp-exp parameterization.
Iter 1/500 - Loss: 13.242
Iter 51/500 - Loss: -2.287
Iter 101/500 - Loss: -2.350
Iter 151/500 - Loss: -2.356
-2.3560118675231934 tensor(-2.3560, device='cuda:6', grad_fn=<NegBackward>)
0.7776696681976318
0.58797287940979
0.37681594491004944
0.14726421236991882
-0.09398385137319565
-0.3399818241596222
-0.5853941440582275
-0.8263618350028992
-1.0599205493927002
-1.2834511995315552
now running  PGR
Using gp-exp parameterization.
Iter 1/500 - Loss: 13.987
Iter 51/500 - Loss: -2.578
Iter 101/500 - Loss: -2.661
Iter 151/500 - Loss: -2.671
-2.6716995239257812 tensor(-2.6717, device='cuda:6', grad_fn=<NegBackward>)
0.774935245513916
0.5812886357307434
0.3638145327568054
0.13096250593662262
-0.1116119921207428
-0.3600108027458191
-0.6085504293441772
-0.8531656861305237
-1.0

Iter 101/500 - Loss: -2.978
Iter 151/500 - Loss: -2.987
-2.9877800941467285 tensor(-2.9878, device='cuda:6', grad_fn=<NegBackward>)
0.7786756157875061
0.5850063562393188
0.36495429277420044
0.12538105249404907
-0.11990369111299515
-0.36242884397506714
-0.5940259099006653
-0.8117339611053467
-1.0375688076019287
-1.2591406106948853
now running  C
Using gp-exp parameterization.
Iter 1/500 - Loss: 14.580
Iter 51/500 - Loss: -2.257
Iter 101/500 - Loss: -2.353
Iter 151/500 - Loss: -2.367
-2.369436025619507 tensor(-2.3694, device='cuda:6', grad_fn=<NegBackward>)
0.7744675278663635
0.5826705694198608
0.3689539134502411
0.1392195075750351
-0.10180506855249405
-0.34780099987983704
-0.593120813369751
-0.8337161540985107
-1.0664399862289429
-1.2884495258331299
now running  CB
Using gp-exp parameterization.
Iter 1/500 - Loss: 12.743
Iter 51/500 - Loss: -2.461
Iter 101/500 - Loss: -2.517
Iter 151/500 - Loss: -2.523
-2.523850440979004 tensor(-2.5238, device='cuda:6', grad_fn=<NegBackward>)
0.77421402

now running  TFC
Using gp-exp parameterization.
Iter 1/500 - Loss: 14.810
Iter 51/500 - Loss: -2.218
Iter 101/500 - Loss: -2.326
Iter 151/500 - Loss: -2.338
-2.3389952182769775 tensor(-2.3390, device='cuda:6', grad_fn=<NegBackward>)
0.773535966873169
0.5816513895988464
0.36777231097221375
0.1376393735408783
-0.10352762043476105
-0.34985825419425964
-0.5956077575683594
-0.8366039991378784
-1.0695840120315552
-1.2916744947433472
now running  TROW
Using gp-exp parameterization.
Iter 1/500 - Loss: 13.621
Iter 51/500 - Loss: -2.524
Iter 101/500 - Loss: -2.598
Iter 151/500 - Loss: -2.603
-2.602607488632202 tensor(-2.6026, device='cuda:6', grad_fn=<NegBackward>)
0.7737215161323547
0.579362690448761
0.35985681414604187
0.12286244332790375
-0.12272366136312485
-0.36908650398254395
-0.6087984442710876
-0.8396288156509399
-1.0694165229797363
-1.2917869091033936
now running  TRV
Using gp-exp parameterization.
Iter 1/500 - Loss: 13.472
Iter 51/500 - Loss: -2.521
Iter 101/500 - Loss: -2.612
Iter 151

In [46]:
quantiles = torch.stack(
    [(x_paths[..., -1] > full_data[:, tend-1].unsqueeze(-1).cpu()
     ).sum(1)/100 for x_paths, tend in zip(x_list, eval_splits)]
)

In [48]:
torch.save(f="finance_ind_preds.pt", obj={"quantiles": quantiles, "x_paths": x_list, "v_paths": v_list,
                                         "y": full_data})